In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate

In [3]:
#IT5
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pyth

In [ ]:
from datasets import Dataset
import json
import pandas as pd
from sklearn.model_selection import train_test_split

data = []

with open('datasetT5.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

# Dividi il DataFrame in set di addestramento e test
train_df, test_df = train_test_split(df, test_size=0.4, random_state=42)

# Converti i DataFrame in oggetti Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


# Definisci la funzione di tokenizzazione
def tokenize_function(examples):
    return tokenizer(examples['source'], padding='max_length', truncation=True )

# Applica la tokenizzazione usando il metodo map
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)



In [5]:
small_train_dataset = tokenized_train_dataset.shuffle(seed=42).select(range(60))
small_eval_dataset = tokenized_test_dataset.shuffle(seed=42).select(range(40))

In [ ]:
!pip install --upgrade transformers

In [ ]:
!pip install transformers[torch] accelerate -U

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
!pip install --upgrade tensorflow==2.15

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [13]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [16]:
from transformers import DataCollatorForLanguageModeling
from torch.utils.data.dataloader import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)


trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        compute_metrics=compute_metrics,

)

TypeError: Trainer.__init__() got an unexpected keyword argument 'input_ids'

In [15]:
trainer.train()

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

# Nuova sezione

In [21]:
#Script that runs the Fine tuning separetely in order to create CUDA statistics
#Needs to be run separate in order to not take too much GPU memory.
#Author: JP Rivera


import torch
import pickle
import argparse
from transformers import TrainingArguments, Trainer
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MT5ForConditionalGeneration, Trainer, TrainingArguments

# Define the compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


def main( train_data_path: str):
    #model = torch.load(model_path)  # Load the model
    #model = AutoModel.from_pretrained(model_path)
    #model = MT5ForConditionalGeneration.from_pretrained(model_path)
    training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
    #IT5
    tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
    model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")



    with open('datasetT5.jsonl', 'r') as f:
        for line in f:
            data.append(json.loads(line))



    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=data,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()



#Main method acctps inpus args
if __name__ == "__main__":
    train_data_path="datasetT5.jsonl"
    main(train_data_path)



/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,decoder_input_ids,decoder_attention_mask,head_mask,decoder_head_mask,cross_attn_head_mask,encoder_outputs,past_key_values,inputs_embeds,decoder_inputs_embeds,labels,use_cache,output_attentions,output_hidden_states,return_dict,labels,label_ids,label.